In [1]:
import os
import time
import datetime
import csv
import json
from math import sqrt
import warnings

import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score
warnings.filterwarnings("ignore")

In [2]:
# 参数配置

class TrainingConfig(object):
    epoches = 10
    evaluateEvery = 100
    checkpointEvery = 100
    learningRate = 0.001
    

class ModelConfig(object):
    
    # 该列表中子列表的三个元素分别是卷积核的数量，卷积核的高度，池化的尺寸
    convLayers = [[256, 7, 4],
                  [256, 7, 4],
                  [256, 3, 4]]
#                   [256, 3, None],
#                   [256, 3, None],
#                   [256, 3, 3]]
    fcLayers = [512]
    dropoutKeepProb = 0.5
    
    epsilon = 1e-3  # BN层中防止分母为0而加入的极小值
    decay = 0.999  # BN层中用来计算滑动平均的值
    
    
class Config(object):
    alphabet = "abcdefghijklmnopqrstuvwxyz0123456789-,;.!?:'\"/\\|_@#$%^&*~`+-=<>()[]{}"
#     alphabet = "abcdefghijklmnopqrstuvwxyz0123456789"
    
    sequenceLength = 1014
    batchSize = 128
    
    rate = 0.8  # 训练集的比例
    
    dataSource = "../data/preProcess/labeledCharTrain.csv"
    
    training = TrainingConfig()
    
    model = ModelConfig()

config = Config()

In [3]:
# 数据预处理的类，生成训练集和测试集

class Dataset(object):
    def __init__(self, config):
        self._dataSource = config.dataSource
        
        self._sequenceLength = config.sequenceLength
        self._rate = config.rate
        
        self.trainReviews = []
        self.trainLabels = []
        
        self.evalReviews = []
        self.evalLabels = []
        
        self._alphabet = config.alphabet
        self.charEmbedding =None
        
        self._charToIndex = {}
        self._indexToChar = {}
        
    def _readData(self, filePath):
        """
        从csv文件中读取数据集
        """
        
        df = pd.read_csv(filePath)
        labels = df["sentiment"].tolist()
        review = df["review"].tolist()
        reviews = [[char for char in line if char != " "] for line in review]
        
        return reviews, labels

    def _reviewProcess(self, review, sequenceLength, charToIndex):
        """
        将数据集中的每条评论用index表示
        wordToIndex中“pad”对应的index为0
        """
        
        reviewVec = np.zeros((sequenceLength))
        sequenceLen = sequenceLength
        
        # 判断当前的序列是否小于定义的固定序列长度
        if len(review) < sequenceLength:
            sequenceLen = len(review)
            
        for i in range(sequenceLen):
            if review[i] in charToIndex:
                reviewVec[i] = charToIndex[review[i]]
            else:
                reviewVec[i] = charToIndex["UNK"]

        return reviewVec

    def _genTrainEvalData(self, x, y, rate):
        """
        生成训练集和验证集
        """
        
        reviews = []
        labels = []
        
        # 遍历所有的文本，将文本中的词转换成index表示
        
        for i in range(len(x)):
            reviewVec = self._reviewProcess(x[i], self._sequenceLength, self._charToIndex)
            reviews.append(reviewVec)
            
            labels.append([y[i]])
            
        trainIndex = int(len(x) * rate)
        
        trainReviews = np.asarray(reviews[:trainIndex], dtype="int64")
        trainLabels = np.array(labels[:trainIndex], dtype="float32")
        
        evalReviews = np.asarray(reviews[trainIndex:], dtype="int64")
        evalLabels = np.array(labels[trainIndex:], dtype="float32")

        return trainReviews, trainLabels, evalReviews, evalLabels
        
    def _genVocabulary(self, reviews):
        """
        生成字符向量和字符-索引映射字典
        """
        
        chars = [char for char in self._alphabet]
        
        vocab, charEmbedding = self._getCharEmbedding(chars)
        self.charEmbedding = charEmbedding
        
        self._charToIndex = dict(zip(vocab, list(range(len(vocab)))))
        self._indexToChar = dict(zip(list(range(len(vocab))), vocab))
        
        # 将词汇-索引映射表保存为json数据，之后做inference时直接加载来处理数据
        with open("../data/charJson/charToIndex.json", "w", encoding="utf-8") as f:
            json.dump(self._charToIndex, f)
        
        with open("../data/charJson/indexToChar.json", "w", encoding="utf-8") as f:
            json.dump(self._indexToChar, f)
            
    def _getCharEmbedding(self, chars):
        """
        按照one的形式将字符映射成向量
        """
        
        alphabet = ["UNK"] + [char for char in self._alphabet]
        vocab = ["pad"] + alphabet
        charEmbedding = []
        charEmbedding.append(np.zeros(len(alphabet), dtype="float32"))
        
        for i, alpha in enumerate(alphabet):
            onehot = np.zeros(len(alphabet), dtype="float32")
            
            # 生成每个字符对应的向量
            onehot[i] = 1
            
            # 生成字符嵌入的向量矩阵
            charEmbedding.append(onehot)
                
        return vocab, np.array(charEmbedding)
            
    def dataGen(self):
        """
        初始化训练集和验证集
        """
        
        # 初始化数据集
        reviews, labels = self._readData(self._dataSource)
        
        # 初始化词汇-索引映射表和词向量矩阵
        self._genVocabulary(reviews)
        
        # 初始化训练集和测试集
        trainReviews, trainLabels, evalReviews, evalLabels = self._genTrainEvalData(reviews, labels, self._rate)
        self.trainReviews = trainReviews
        self.trainLabels = trainLabels
        
        self.evalReviews = evalReviews
        self.evalLabels = evalLabels
        
        
data = Dataset(config)
data.dataGen()

In [4]:
print("train data shape: {}".format(data.trainReviews.shape))
print("train label shape: {}".format(data.trainLabels.shape))
print("eval data shape: {}".format(data.evalReviews.shape))
print("charEmbedding shape: {}".format(data.charEmbedding.shape))

train data shape: (20000, 1014)
train label shape: (20000, 1)
eval data shape: (5000, 1014)
charEmbedding shape: (71, 70)


In [5]:
# 输出batch数据集

def nextBatch(x, y, batchSize):
        """
        生成batch数据集，用生成器的方式输出
        """
    
        perm = np.arange(len(x))
        np.random.shuffle(perm)
        x = x[perm]
        y = y[perm]
        
        numBatches = len(x) // batchSize

        for i in range(numBatches):
            start = i * batchSize
            end = start + batchSize
            batchX = np.array(x[start: end], dtype="int64")
            batchY = np.array(y[start: end], dtype="float32")
            
            yield batchX, batchY

In [6]:
# 定义char-CNN分类器

class CharCNN(object):
    """
    char-CNN用于文本分类
    """
    def __init__(self, config, charEmbedding):
        # placeholders for input, output and dropuot
        self.inputX = tf.placeholder(tf.int32, [None, config.sequenceLength], name="inputX")
        self.inputY = tf.placeholder(tf.float32, [None, 1], name="inputY")
        self.dropoutKeepProb = tf.placeholder(tf.float32, name="dropoutKeepProb")
        self.isTraining = tf.placeholder(tf.bool, name="isTraining")
        
        self.epsilon = config.model.epsilon
        self.decay = config.model.decay
        
        # 字符嵌入
        with tf.name_scope("embedding"):
            
            # 利用one-hot的字符向量作为初始化词嵌入矩阵
            self.W = tf.Variable(tf.cast(charEmbedding, dtype=tf.float32, name="charEmbedding") ,name="W")
            # 获得字符嵌入
            self.embededChars = tf.nn.embedding_lookup(self.W, self.inputX)
            # 添加一个通道维度
            self.embededCharsExpand = tf.expand_dims(self.embededChars, -1)

        for i, cl in enumerate(config.model.convLayers):
            print("开始第" + str(i + 1) + "卷积层的处理")
            # 利用命名空间name_scope来实现变量名复用
            with tf.name_scope("convLayer-%s"%(i+1)):
                # 获取字符的向量长度
                filterWidth = self.embededCharsExpand.get_shape()[2].value
                
                # filterShape = [height, width, in_channels, out_channels]
                filterShape = [cl[1], filterWidth, 1, cl[0]]

                stdv = 1 / sqrt(cl[0] * cl[1])
                
                # 初始化w和b的值
                wConv = tf.Variable(tf.random_uniform(filterShape, minval=-stdv, maxval=stdv),
                                     dtype='float32', name='w')
                bConv = tf.Variable(tf.random_uniform(shape=[cl[0]], minval=-stdv, maxval=stdv), name='b')
                
#                 w_conv = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.05), name="w")
#                 b_conv = tf.Variable(tf.constant(0.1, shape=[cl[0]]), name="b")
                # 构建卷积层，可以直接将卷积核的初始化方法传入（w_conv）
                conv = tf.nn.conv2d(self.embededCharsExpand, wConv, strides=[1, 1, 1, 1], padding="VALID", name="conv")
                # 加上偏差
                hConv = tf.nn.bias_add(conv, bConv)
                # 可以直接加上relu函数，因为tf.nn.conv2d事实上是做了一个卷积运算，然后在这个运算结果上加上偏差，再导入到relu函数中
                hConv = tf.nn.relu(hConv)
                
#                 with tf.name_scope("batchNormalization"):
#                     hConvBN = self._batchNorm(hConv)
                
                if cl[-1] is not None:
                    ksizeShape = [1, cl[2], 1, 1]
                    hPool = tf.nn.max_pool(hConv, ksize=ksizeShape, strides=ksizeShape, padding="VALID", name="pool")
                else:
                    hPool = hConv
                    
                print(hPool.shape)
    
                # 对维度进行转换，转换成卷积层的输入维度
                self.embededCharsExpand = tf.transpose(hPool, [0, 1, 3, 2], name="transpose")
        print(self.embededCharsExpand)
        with tf.name_scope("reshape"):
            fcDim = self.embededCharsExpand.get_shape()[1].value * self.embededCharsExpand.get_shape()[2].value
            self.inputReshape = tf.reshape(self.embededCharsExpand, [-1, fcDim])
        
        # 保存的是神经元的个数[34*256, 1024, 1024]
        weights = [fcDim] + config.model.fcLayers
        
        for i, fl in enumerate(config.model.fcLayers):
            with tf.name_scope("fcLayer-%s"%(i+1)):
                print("开始第" + str(i + 1) + "全连接层的处理")
                stdv = 1 / sqrt(weights[i])
                
                # 定义全连接层的初始化方法，均匀分布初始化w和b的值
                wFc = tf.Variable(tf.random_uniform([weights[i], fl], minval=-stdv, maxval=stdv), dtype="float32", name="w")
                bFc = tf.Variable(tf.random_uniform(shape=[fl], minval=-stdv, maxval=stdv), dtype="float32", name="b")
                
#                 w_fc = tf.Variable(tf.truncated_normal([weights[i], fl], stddev=0.05), name="W")
#                 b_fc = tf.Variable(tf.constant(0.1, shape=[fl]), name="b")
                
                self.fcInput = tf.nn.relu(tf.matmul(self.inputReshape, wFc) + bFc)
                
                with tf.name_scope("dropOut"):
                    self.fcInputDrop = tf.nn.dropout(self.fcInput, self.dropoutKeepProb)
                    
            self.inputReshape = self.fcInputDrop
            
        with tf.name_scope("outputLayer"):
            stdv = 1 / sqrt(weights[-1])
            # 定义隐层到输出层的权重系数和偏差的初始化方法
#             w_out = tf.Variable(tf.truncated_normal([fc_layers[-1], num_classes], stddev=0.1), name="W")
#             b_out = tf.Variable(tf.constant(0.1, shape=[num_classes]), name="b")
            
            wOut = tf.Variable(tf.random_uniform([config.model.fcLayers[-1], 1], minval=-stdv, maxval=stdv), dtype="float32", name="w")
            bOut = tf.Variable(tf.random_uniform(shape=[1], minval=-stdv, maxval=stdv), name="b")
            # tf.nn.xw_plus_b就是x和w的乘积加上b
            self.predictions = tf.nn.xw_plus_b(self.inputReshape, wOut, bOut, name="predictions")
            # 进行二元分类
            self.binaryPreds = tf.cast(tf.greater_equal(self.predictions, 0.5), tf.float32, name="binaryPreds")
            
            
        with tf.name_scope("loss"):
            # 定义损失函数，对预测值进行softmax，再求交叉熵。
            
            losses = tf.nn.sigmoid_cross_entropy_with_logits(logits=self.predictions, labels=self.inputY)
            self.loss = tf.reduce_mean(losses)
    
    def _batchNorm(self, x):
        
        gamma = tf.Variable(tf.ones([x.get_shape()[3].value]))
        beta = tf.Variable(tf.zeros([x.get_shape()[3].value]))

        self.popMean = tf.Variable(tf.zeros([x.get_shape()[3].value]), trainable=False, name="popMean")
        self.popVariance = tf.Variable(tf.ones([x.get_shape()[3].value]), trainable=False, name="popVariance")

        def batchNormTraining():
            # 一定要使用正确的维度确保计算的是每个特征图上的平均值和方差而不是整个网络节点上的统计分布值
            batchMean, batchVariance = tf.nn.moments(x, [0, 1, 2], keep_dims=False)

            decay = 0.99
            trainMean = tf.assign(self.popMean, self.popMean*self.decay + batchMean*(1 - self.decay))
            trainVariance = tf.assign(self.popVariance, self.popVariance*self.decay + batchVariance*(1 - self.decay))

            with tf.control_dependencies([trainMean, trainVariance]):
                return tf.nn.batch_normalization(x, batchMean, batchVariance, beta, gamma, self.epsilon)

        def batchNormInference():
            return tf.nn.batch_normalization(x, self.popMean, self.popVariance, beta, gamma, self.epsilon)

        batchNormalizedOutput = tf.cond(self.isTraining, batchNormTraining, batchNormInference)
        return tf.nn.relu(batchNormalizedOutput)

In [7]:
# 定义性能指标函数

def mean(item):
    return sum(item) / len(item)


def genMetrics(trueY, predY, binaryPredY):
    """
    生成acc和auc值
    """
    
    auc = roc_auc_score(trueY, predY)
    accuracy = accuracy_score(trueY, binaryPredY)
    precision = precision_score(trueY, binaryPredY, average='macro')
    recall = recall_score(trueY, binaryPredY, average='macro')
    
    return round(accuracy, 4), round(auc, 4), round(precision, 4), round(recall, 4)

In [8]:
# 训练模型

# 生成训练集和验证集
trainReviews = data.trainReviews
trainLabels = data.trainLabels
evalReviews = data.evalReviews
evalLabels = data.evalLabels

charEmbedding = data.charEmbedding

# 定义计算图
with tf.Graph().as_default():

    session_conf = tf.ConfigProto(allow_soft_placement=True, log_device_placement=False)
    session_conf.gpu_options.allow_growth=True
    session_conf.gpu_options.per_process_gpu_memory_fraction = 0.9  # 配置gpu占用率  

    sess = tf.Session(config=session_conf)
    
    # 定义会话
    with sess.as_default():
        
        cnn = CharCNN(config, charEmbedding)
        globalStep = tf.Variable(0, name="globalStep", trainable=False)
        # 定义优化函数，传入学习速率参数
        optimizer = tf.train.RMSPropOptimizer(config.training.learningRate)
        # 计算梯度,得到梯度和变量
        gradsAndVars = optimizer.compute_gradients(cnn.loss)
        # 将梯度应用到变量下，生成训练器
        trainOp = optimizer.apply_gradients(gradsAndVars, global_step=globalStep)
        
        # 用summary绘制tensorBoard
        gradSummaries = []
        for g, v in gradsAndVars:
            if g is not None:
                tf.summary.histogram("{}/grad/hist".format(v.name), g)
                tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
        
        outDir = os.path.abspath(os.path.join(os.path.curdir, "summarys"))
        print("Writing to {}\n".format(outDir))
        
        lossSummary = tf.summary.scalar("trainLoss", cnn.loss)
        
        summaryOp = tf.summary.merge_all()
        
        trainSummaryDir = os.path.join(outDir, "train")
        trainSummaryWriter = tf.summary.FileWriter(trainSummaryDir, sess.graph)
        
        evalSummaryDir = os.path.join(outDir, "eval")
        evalSummaryWriter = tf.summary.FileWriter(evalSummaryDir, sess.graph)
        
        
        # 初始化所有变量
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=5)
        
        # 保存模型的一种方式，保存为pb文件
#         builder = tf.saved_model.builder.SavedModelBuilder("../model/charCNN/savedModel")
        sess.run(tf.global_variables_initializer())

        def trainStep(batchX, batchY):
            """
            训练函数
            """   
            feed_dict = {
              cnn.inputX: batchX,
              cnn.inputY: batchY,
              cnn.dropoutKeepProb: config.model.dropoutKeepProb,
              cnn.isTraining: True
            }
            _, summary, step, loss, predictions, binaryPreds = sess.run(
                [trainOp, summaryOp, globalStep, cnn.loss, cnn.predictions, cnn.binaryPreds],
                feed_dict)
            timeStr = datetime.datetime.now().isoformat()
            acc, auc, precision, recall = genMetrics(batchY, predictions, binaryPreds)
            print("{}, step: {}, loss: {}, acc: {}, auc: {}, precision: {}, recall: {}".format(timeStr, step, loss, acc, auc, precision, recall))
            trainSummaryWriter.add_summary(summary, step)

        def devStep(batchX, batchY):
            """
            验证函数
            """
            feed_dict = {
              cnn.inputX: batchX,
              cnn.inputY: batchY,
              cnn.dropoutKeepProb: 1.0,
              cnn.isTraining: False
            }
            summary, step, loss, predictions, binaryPreds = sess.run(
                [summaryOp, globalStep, cnn.loss, cnn.predictions, cnn.binaryPreds],
                feed_dict)
            
            acc, auc, precision, recall = genMetrics(batchY, predictions, binaryPreds)
            
            evalSummaryWriter.add_summary(summary, step)
            
            return loss, acc, auc, precision, recall
        
        for i in range(config.training.epoches):
            # 训练模型
            print("start training model")
            for batchTrain in nextBatch(trainReviews, trainLabels, config.batchSize):
                trainStep(batchTrain[0], batchTrain[1])

                currentStep = tf.train.global_step(sess, globalStep) 
                if currentStep % config.training.evaluateEvery == 0:
                    print("\nEvaluation:")
                    
                    losses = []
                    accs = []
                    aucs = []
                    precisions = []
                    recalls = []
                    
                    for batchEval in nextBatch(evalReviews, evalLabels, config.batchSize):
                        loss, acc, auc, precision, recall = devStep(batchEval[0], batchEval[1])
                        losses.append(loss)
                        accs.append(acc)
                        aucs.append(auc)
                        precisions.append(precision)
                        recalls.append(recall)
                        
                    time_str = datetime.datetime.now().isoformat()
                    print("{}, step: {}, loss: {}, acc: {}, auc: {}, precision: {}, recall: {}".format(time_str, currentStep, mean(losses), 
                                                                                                       mean(accs), mean(aucs), mean(precisions),
                                                                                                       mean(recalls)))
                    
#                 if currentStep % config.training.checkpointEvery == 0:
#                     # 保存模型的另一种方法，保存checkpoint文件
#                     path = saver.save(sess, "../model/charCNN/model/my-model", global_step=currentStep)
#                     print("Saved model checkpoint to {}\n".format(path))
                    
#         inputs = {"inputX": tf.saved_model.utils.build_tensor_info(cnn.inputX),
#                   "keepProb": tf.saved_model.utils.build_tensor_info(cnn.dropoutKeepProb)}

#         outputs = {"binaryPreds": tf.saved_model.utils.build_tensor_info(cnn.binaryPreds)}

#         prediction_signature = tf.saved_model.signature_def_utils.build_signature_def(inputs=inputs, outputs=outputs,
#                                                                                       method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME)
#         legacy_init_op = tf.group(tf.tables_initializer(), name="legacy_init_op")
#         builder.add_meta_graph_and_variables(sess, [tf.saved_model.tag_constants.SERVING],
#                                             signature_def_map={"predict": prediction_signature}, legacy_init_op=legacy_init_op)

#         builder.save()

开始第1卷积层的处理
(?, 252, 1, 256)
开始第2卷积层的处理
(?, 61, 1, 256)
开始第3卷积层的处理
(?, 14, 1, 256)
Tensor("convLayer-3/transpose:0", shape=(?, 14, 256, 1), dtype=float32)
开始第1全连接层的处理
INFO:tensorflow:Summary name embedding/W:0/grad/hist is illegal; using embedding/W_0/grad/hist instead.
INFO:tensorflow:Summary name embedding/W:0/grad/sparsity is illegal; using embedding/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name convLayer-1/w:0/grad/hist is illegal; using convLayer-1/w_0/grad/hist instead.
INFO:tensorflow:Summary name convLayer-1/w:0/grad/sparsity is illegal; using convLayer-1/w_0/grad/sparsity instead.
INFO:tensorflow:Summary name convLayer-1/b:0/grad/hist is illegal; using convLayer-1/b_0/grad/hist instead.
INFO:tensorflow:Summary name convLayer-1/b:0/grad/sparsity is illegal; using convLayer-1/b_0/grad/sparsity instead.
INFO:tensorflow:Summary name convLayer-2/w:0/grad/hist is illegal; using convLayer-2/w_0/grad/hist instead.
INFO:tensorflow:Summary name convLayer-2/w:0/grad/sparsity is 

2018-12-29T17:20:13.381719, step: 47, loss: 0.6948320865631104, acc: 0.5938, auc: 0.5281, precision: 0.2969, recall: 0.5
2018-12-29T17:20:13.480149, step: 48, loss: 0.6928154230117798, acc: 0.5078, auc: 0.6037, precision: 0.2539, recall: 0.5
2018-12-29T17:20:13.585767, step: 49, loss: 0.6941088438034058, acc: 0.5547, auc: 0.5147, precision: 0.2773, recall: 0.5
2018-12-29T17:20:13.679185, step: 50, loss: 0.6926795840263367, acc: 0.5, auc: 0.5974, precision: 0.25, recall: 0.5
2018-12-29T17:20:13.783856, step: 51, loss: 0.692560076713562, acc: 0.4531, auc: 0.4754, precision: 0.2266, recall: 0.5
2018-12-29T17:20:13.890484, step: 52, loss: 0.694084107875824, acc: 0.5547, auc: 0.5028, precision: 0.2773, recall: 0.5
2018-12-29T17:20:13.990723, step: 53, loss: 0.6924004554748535, acc: 0.4688, auc: 0.5532, precision: 0.2344, recall: 0.5
2018-12-29T17:20:14.093485, step: 54, loss: 0.6915577054023743, acc: 0.4297, auc: 0.5841, precision: 0.2148, recall: 0.5
2018-12-29T17:20:14.195967, step: 55, l

2018-12-29T17:20:24.103917, step: 116, loss: 0.6937824487686157, acc: 0.4688, auc: 0.4775, precision: 0.2344, recall: 0.5
2018-12-29T17:20:24.196825, step: 117, loss: 0.6926805973052979, acc: 0.5234, auc: 0.5566, precision: 0.2617, recall: 0.5
2018-12-29T17:20:24.288063, step: 118, loss: 0.6930062174797058, acc: 0.5312, auc: 0.4733, precision: 0.2656, recall: 0.5
2018-12-29T17:20:24.390876, step: 119, loss: 0.6936447620391846, acc: 0.4531, auc: 0.5567, precision: 0.2266, recall: 0.5
2018-12-29T17:20:24.485181, step: 120, loss: 0.6934823989868164, acc: 0.4375, auc: 0.4973, precision: 0.2188, recall: 0.5
2018-12-29T17:20:24.578189, step: 121, loss: 0.6930456161499023, acc: 0.4766, auc: 0.4837, precision: 0.2383, recall: 0.5
2018-12-29T17:20:24.672182, step: 122, loss: 0.6933823823928833, acc: 0.5391, auc: 0.5633, precision: 0.2695, recall: 0.5
2018-12-29T17:20:24.787839, step: 123, loss: 0.6927562952041626, acc: 0.5078, auc: 0.5665, precision: 0.2539, recall: 0.5
2018-12-29T17:20:24.8798

2018-12-29T17:20:30.269838, step: 184, loss: 0.6912068724632263, acc: 0.5547, auc: 0.5135, precision: 0.2773, recall: 0.5
2018-12-29T17:20:30.348206, step: 185, loss: 0.695985734462738, acc: 0.4766, auc: 0.5872, precision: 0.2383, recall: 0.5
2018-12-29T17:20:30.427008, step: 186, loss: 0.6949476003646851, acc: 0.4375, auc: 0.4945, precision: 0.2188, recall: 0.5
2018-12-29T17:20:30.507933, step: 187, loss: 0.6938637495040894, acc: 0.5391, auc: 0.4864, precision: 0.2695, recall: 0.5
2018-12-29T17:20:30.589769, step: 188, loss: 0.6930268406867981, acc: 0.5078, auc: 0.5009, precision: 0.2539, recall: 0.5
2018-12-29T17:20:30.678140, step: 189, loss: 0.6932500600814819, acc: 0.5, auc: 0.4932, precision: 0.25, recall: 0.5
2018-12-29T17:20:30.757477, step: 190, loss: 0.6936135292053223, acc: 0.4922, auc: 0.4347, precision: 0.2461, recall: 0.5
2018-12-29T17:20:30.844656, step: 191, loss: 0.6932598352432251, acc: 0.4922, auc: 0.4672, precision: 0.2461, recall: 0.5
2018-12-29T17:20:30.925605, st

2018-12-29T17:20:38.829703, step: 251, loss: 0.693954348564148, acc: 0.4453, auc: 0.5036, precision: 0.2227, recall: 0.5
2018-12-29T17:20:38.906244, step: 252, loss: 0.6944657564163208, acc: 0.5469, auc: 0.4855, precision: 0.2734, recall: 0.5
2018-12-29T17:20:38.981030, step: 253, loss: 0.6930980086326599, acc: 0.4297, auc: 0.5046, precision: 0.2148, recall: 0.5
2018-12-29T17:20:39.058927, step: 254, loss: 0.6939687132835388, acc: 0.5547, auc: 0.5246, precision: 0.2773, recall: 0.5
2018-12-29T17:20:39.154522, step: 255, loss: 0.6924185752868652, acc: 0.5078, auc: 0.5529, precision: 0.2539, recall: 0.5
2018-12-29T17:20:39.236660, step: 256, loss: 0.6932753920555115, acc: 0.5312, auc: 0.4824, precision: 0.2656, recall: 0.5
2018-12-29T17:20:39.337078, step: 257, loss: 0.6939822435379028, acc: 0.4375, auc: 0.5258, precision: 0.2188, recall: 0.5
2018-12-29T17:20:39.414235, step: 258, loss: 0.6936004757881165, acc: 0.5078, auc: 0.4684, precision: 0.2539, recall: 0.5
2018-12-29T17:20:39.49470

...

2018-12-29T17:22:41.677110, step: 1246, loss: 0.40080562233924866, acc: 0.7969, auc: 0.9448, precision: 0.8298, recall: 0.7944
2018-12-29T17:22:41.780573, step: 1247, loss: 0.2741144895553589, acc: 0.9062, auc: 0.9704, precision: 0.9067, recall: 0.9075
2018-12-29T17:22:41.870390, step: 1248, loss: 0.21627333760261536, acc: 0.8828, auc: 0.9773, precision: 0.896, recall: 0.88
start training model
2018-12-29T17:22:42.125732, step: 1249, loss: 0.2784898281097412, acc: 0.8906, auc: 0.9515, precision: 0.8911, recall: 0.8892
2018-12-29T17:22:42.217230, step: 1250, loss: 0.1974686235189438, acc: 0.9062, auc: 0.9765, precision: 0.9107, recall: 0.9047
2018-12-29T17:22:42.310156, step: 1251, loss: 0.1769554615020752, acc: 0.9219, auc: 0.9834, precision: 0.9223, recall: 0.9232
2018-12-29T17:22:42.410429, step: 1252, loss: 0.20097583532333374, acc: 0.9219, auc: 0.9771, precision: 0.9195, recall: 0.9195
2018-12-29T17:22:42.497746, step: 1253, loss: 0.12877146899700165, acc: 0.9531, auc: 0.9939, prec

2018-12-29T17:22:52.053770, step: 1311, loss: 0.35674506425857544, acc: 0.7969, auc: 0.9976, precision: 0.8539, recall: 0.8
2018-12-29T17:22:52.146945, step: 1312, loss: 0.2676755487918854, acc: 0.8984, auc: 0.9634, precision: 0.8998, recall: 0.896
2018-12-29T17:22:52.234851, step: 1313, loss: 0.20909716188907623, acc: 0.9297, auc: 0.9748, precision: 0.9317, recall: 0.9249
2018-12-29T17:22:52.341577, step: 1314, loss: 0.20614613592624664, acc: 0.9062, auc: 0.9741, precision: 0.9081, recall: 0.9057
2018-12-29T17:22:52.436052, step: 1315, loss: 0.18375937640666962, acc: 0.9453, auc: 0.9819, precision: 0.9431, recall: 0.9474
2018-12-29T17:22:52.528485, step: 1316, loss: 0.1879454255104065, acc: 0.9141, auc: 0.9856, precision: 0.9286, recall: 0.9113
2018-12-29T17:22:52.622997, step: 1317, loss: 0.13838538527488708, acc: 0.9453, auc: 0.9914, precision: 0.9446, recall: 0.9456
2018-12-29T17:22:52.718237, step: 1318, loss: 0.23226581513881683, acc: 0.8906, auc: 0.9814, precision: 0.9058, recal

2018-12-29T17:22:58.564967, step: 1377, loss: 0.1896437108516693, acc: 0.9375, auc: 0.9772, precision: 0.9383, recall: 0.9366
2018-12-29T17:22:58.659867, step: 1378, loss: 0.26517921686172485, acc: 0.9062, auc: 0.9617, precision: 0.9118, recall: 0.9118
2018-12-29T17:22:58.756426, step: 1379, loss: 0.3498218059539795, acc: 0.9062, auc: 0.9709, precision: 0.918, recall: 0.9037
2018-12-29T17:22:58.848515, step: 1380, loss: 0.48902446031570435, acc: 0.7031, auc: 0.9617, precision: 0.8155, recall: 0.6984
2018-12-29T17:22:58.938750, step: 1381, loss: 0.34719228744506836, acc: 0.8984, auc: 0.9657, precision: 0.9042, recall: 0.9015
2018-12-29T17:22:59.037344, step: 1382, loss: 0.20789608359336853, acc: 0.9062, auc: 0.9742, precision: 0.9089, recall: 0.9008
2018-12-29T17:22:59.126120, step: 1383, loss: 0.1357952058315277, acc: 0.9688, auc: 0.995, precision: 0.968, recall: 0.968
2018-12-29T17:22:59.228418, step: 1384, loss: 0.2738726735115051, acc: 0.8594, auc: 0.9597, precision: 0.8684, recall:

2018-12-29T17:23:08.902393, step: 1441, loss: 0.16603147983551025, acc: 0.9453, auc: 0.9889, precision: 0.9427, recall: 0.945
2018-12-29T17:23:08.995018, step: 1442, loss: 0.17771635949611664, acc: 0.8906, auc: 0.9932, precision: 0.8971, recall: 0.9054
2018-12-29T17:23:09.088320, step: 1443, loss: 0.16966921091079712, acc: 0.9453, auc: 0.9886, precision: 0.9436, recall: 0.9472
2018-12-29T17:23:09.194234, step: 1444, loss: 0.17125730216503143, acc: 0.9141, auc: 0.9792, precision: 0.917, recall: 0.9134
2018-12-29T17:23:09.291902, step: 1445, loss: 0.1268976926803589, acc: 0.9375, auc: 0.9914, precision: 0.937, recall: 0.9382
2018-12-29T17:23:09.388774, step: 1446, loss: 0.10459314286708832, acc: 0.9453, auc: 0.9938, precision: 0.9476, recall: 0.9426
2018-12-29T17:23:09.481009, step: 1447, loss: 0.09073600172996521, acc: 0.9609, auc: 0.9956, precision: 0.9637, recall: 0.9579
2018-12-29T17:23:09.584081, step: 1448, loss: 0.09973230212926865, acc: 0.9453, auc: 0.9958, precision: 0.9453, rec

2018-12-29T17:23:19.282621, step: 1506, loss: 0.3198937773704529, acc: 0.8672, auc: 0.977, precision: 0.8896, recall: 0.875
2018-12-29T17:23:19.380293, step: 1507, loss: 0.17050494253635406, acc: 0.9531, auc: 0.9955, precision: 0.9508, recall: 0.9589
2018-12-29T17:23:19.479520, step: 1508, loss: 0.18570426106452942, acc: 0.9219, auc: 0.9846, precision: 0.9231, recall: 0.9315
2018-12-29T17:23:19.576861, step: 1509, loss: 0.22667500376701355, acc: 0.9297, auc: 0.9823, precision: 0.9378, recall: 0.925
2018-12-29T17:23:19.672448, step: 1510, loss: 0.28458935022354126, acc: 0.8125, auc: 0.9934, precision: 0.8667, recall: 0.8065
2018-12-29T17:23:19.767398, step: 1511, loss: 0.2893804907798767, acc: 0.8828, auc: 0.9741, precision: 0.9062, recall: 0.881
2018-12-29T17:23:19.869476, step: 1512, loss: 0.13394753634929657, acc: 0.9297, auc: 0.9938, precision: 0.928, recall: 0.9296
2018-12-29T17:23:19.967374, step: 1513, loss: 0.1370251327753067, acc: 0.9297, auc: 0.991, precision: 0.94, recall: 0.